## Sources:

Zip code link: https://gis-mdc.opendata.arcgis.com/datasets/fee863cb3da0417fa8b5aaf6b671f8a7/explore

## Imports

In [2]:
import pandas as pd
import geopandas as gpd
import altair as alt
import json
import folium
from folium.plugins import FeatureGroupSubGroup
import os

## Data read-in

In [97]:
df = pd.read_csv('HPI_Projections_By_Florida_County - Sheet1.csv')

gdf = gpd.read_file('USA_Counties_Generalized_Boundaries.geojson', geometry='geometry')
gdf = gdf[gdf['STATE_NAME'] == 'Florida']

In [102]:
gdf[gdf['NAME'].str.contains('Johns')]

,OBJECTID,NAME,STATE_NAME,STATE_FIPS,FIPS,SQMI,POPULATION,POP_SQMI,STATE_ABBR,COUNTY_FIPS,geometry
371,372,Saint Johns,Florida,12,12109,611.14,273425,447.4,FL,109,"POLYGON ((-81.52001 29.62596, -81.52065 29.754..."


In [103]:
df.head(1)

,County,1 year forecast,2 year forecast,3 year forecast,4 year forecast,5 year forecast
0,Alachua,2.90%,6.90%,11.10%,15.30%,19.90%


In [101]:
gdf['NAME'] = gdf['NAME'].str.replace(' County', '', regex=False)
gdf['NAME'] = gdf['NAME'].str.replace('St.', 'Saint', regex=False)

In [104]:
gdf = pd.merge(df,gdf,left_on='County',right_on='NAME')

In [105]:
gdf

,County,1 year forecast,2 year forecast,3 year forecast,4 year forecast,5 year forecast,OBJECTID,NAME,STATE_NAME,STATE_FIPS,FIPS,SQMI,POPULATION,POP_SQMI,STATE_ABBR,COUNTY_FIPS,geometry
0,Alachua,2.90%,6.90%,11.10%,15.30%,19.90%,318,Alachua,Florida,12,12001,968.81,278468,287.4,FL,001,"POLYGON ((-82.40526 29.48510, -82.55477 29.481..."
1,Baker,4.70%,10.00%,16.20%,22.50%,29.10%,319,Baker,Florida,12,12003,588.97,28259,48.0,FL,003,"POLYGON ((-82.04629 30.13971, -82.14505 30.140..."
2,Bay,4.50%,9.70%,15.30%,20.80%,26.20%,320,Bay,Florida,12,12005,765.78,175216,228.8,FL,005,"MULTIPOLYGON (((-85.38475 30.57410, -85.38210 ..."
3,Brevard,1.70%,4.70%,8.70%,12.80%,17.00%,322,Brevard,Florida,12,12009,1051.77,606612,576.8,FL,009,"MULTIPOLYGON (((-80.78566 28.78519, -80.76242 ..."
4,Broward,2.50%,6.40%,10.70%,15.00%,20.00%,323,Broward,Florida,12,12011,1221.39,1944375,1591.9,FL,011,"POLYGON ((-80.87399 25.98326, -80.88158 26.258..."
5,Charlotte,5.00%,9.80%,14.70%,19.60%,24.60%,325,Charlotte,Florida,12,12015,710.32,186847,263.0,FL,015,"MULTIPOLYGON (((-82.05483 27.03164, -81.99285 ..."
6,Citrus,3.90%,7.90%,11.80%,15.70%,19.40%,326,Citrus,Florida,12,12017,622.51,153843,247.1,FL,017,"POLYGON ((-82.26534 28.66208, -82.41324 28.662..."
7,Clay,3.80%,7.90%,12.70%,17.40%,22.40%,327,Clay,Florida,12,12019,619.21,218245,352.5,FL,019,"POLYGON ((-82.04629 30.13971, -82.04792 30.189..."
8,Collier,5.40%,11.20%,16.60%,21.90%,27.30%,328,Collier,Florida,12,12021,2030.89,375752,185.0,FL,021,"POLYGON ((-80.87399 25.98326, -80.87454 25.801..."
9,Columbia,3.70%,8.20%,13.00%,17.80%,22.70%,329,Columbia,Florida,12,12023,801.32,69698,87.0,FL,023,"POLYGON ((-82.78749 30.32990, -82.74747 30.323..."


In [106]:
# Now, you can check for valid geometries
gdf = gdf[gdf.geometry.notnull()]

## Data Prepare

In [107]:
gdf.columns

Index(['County', '1 year forecast', '2 year forecast', '3 year forecast',
       '4 year forecast', '5 year forecast', 'OBJECTID', 'NAME', 'STATE_NAME',
       'STATE_FIPS', 'FIPS', 'SQMI', 'POPULATION', 'POP_SQMI', 'STATE_ABBR',
       'COUNTY_FIPS', 'geometry'],
      dtype='object')

In [108]:
gdf = gpd.GeoDataFrame(gdf)

# Filter out GeometryCollection and other unsupported geometries using geom_type
gdf = gdf[gdf.geometry.geom_type.isin(['Polygon', 'MultiPolygon'])]

In [109]:
gdf.columns

Index(['County', '1 year forecast', '2 year forecast', '3 year forecast',
       '4 year forecast', '5 year forecast', 'OBJECTID', 'NAME', 'STATE_NAME',
       'STATE_FIPS', 'FIPS', 'SQMI', 'POPULATION', 'POP_SQMI', 'STATE_ABBR',
       'COUNTY_FIPS', 'geometry'],
      dtype='object')

In [31]:
gdf = gdf.drop(columns=['OBJECTID'])

In [111]:
gdf.to_file('HPI_FloridaCounty.geojson', driver='GeoJSON')

In [112]:
gdf.columns

Index(['County', '1 year forecast', '2 year forecast', '3 year forecast',
       '4 year forecast', '5 year forecast', 'OBJECTID', 'NAME', 'STATE_NAME',
       'STATE_FIPS', 'FIPS', 'SQMI', 'POPULATION', 'POP_SQMI', 'STATE_ABBR',
       'COUNTY_FIPS', 'geometry'],
      dtype='object')

In [113]:
gdf.head(1)

,County,1 year forecast,2 year forecast,3 year forecast,4 year forecast,5 year forecast,OBJECTID,NAME,STATE_NAME,STATE_FIPS,FIPS,SQMI,POPULATION,POP_SQMI,STATE_ABBR,COUNTY_FIPS,geometry
0,Alachua,2.90%,6.90%,11.10%,15.30%,19.90%,318,Alachua,Florida,12,12001,968.81,278468,287.4,FL,001,"POLYGON ((-82.40526 29.48510, -82.55477 29.481..."


In [114]:
gdf_export = gdf.drop(columns=['geometry','OBJECTID','STATE_FIPS','SQMI',\
                               'POPULATION','POP_SQMI','STATE_ABBR','COUNTY_FIPS'])

In [115]:
gdf_export.to_csv('gdf_export.csv')

In [36]:
# import folium

# # Initialize the map centered around Miami-Dade County
# m = folium.Map(location=[25.7617, -80.1918], zoom_start=10)

# # Define a function to create a style_function based on a property
# def create_style_function(property_name):
#     def style_function(feature):
#         # Default fillColor
#         fill_color = '#ffff00'  # Bright yellow for visibility
        
#         try:
#             # Assuming the property value is a string that represents a float
#             value = float(feature['properties'][property_name])
#             if value > 0.1:  # Adjust threshold as necessary
#                 fill_color = '#00ff00'  # Green
#             else:
#                 fill_color = '#ff0000'  # Red
#         except (ValueError, KeyError):
#             # If the value cannot be converted to float or key doesn't exist
#             # The default fill_color will be used
#             pass
        
#         return {
#             'fillColor': fill_color,
#             'color': 'black',  # Outline color
#             'weight': 1,  # Outline weight
#             'fillOpacity': 0.6,
#         }
#     return style_function


# # List of properties you want to toggle as layers
# properties = ['5 Year HPI Change', '1 year forecast (23-24)',
#        '2 year forecast (23-25)', '3 year forecast (23-26)',
#        '4 year forecast (23-27)', '5 year forecast (23-28)']  # Replace with your actual property names

# # Add a GeoJson layer for each property
# for property_name in properties:
#     geojson_layer = folium.GeoJson(
#         'HPI_Miami-Dade.geojson',
#         name=property_name,
#         style_function=create_style_function(property_name),
#         tooltip=folium.GeoJsonTooltip(fields=['zip_code', property_name])
#     ).add_to(m)

# # Add layer control to the map
# folium.LayerControl().add_to(m)

# # Save the map
# # m.save('index.html')
# m

In [37]:
# import folium
# import branca.colormap as cm
# import geopandas as gpd

# # Load your GeoDataFrame
# gdf = gpd.read_file('HPI_Miami-Dade.geojson')

# gdf = gdf[gdf['geometry'].is_valid]

# # Initialize the Folium map with a center point
# m = folium.Map(location=[25.7617, -80.1918], zoom_start=10)

# # Define a function to create a colormap and style function based on the column
# def add_layer(gdf, column_name, map_object):
#     # Determine the range of your data values for the specific column
#     min_value = gdf[column_name].min()
#     max_value = gdf[column_name].max()
    
#     # Create a linear colormap
#     colormap = cm.linear.YlOrRd_09.scale(min_value, max_value)
#     colormap.caption = column_name  # Set a caption for the color scale bar
    
#     # Define the style function using the colormap
#     def style_function(feature):
#         value = feature['properties'][column_name]
#         return {
#             'fillColor': colormap(value) if value is not None else 'transparent',
#             'color': 'black',  # Outline color
#             'weight': 1,  # Outline weight
#             'fillOpacity': 0.6,
#         }
    
#     # Create a GeoJson layer with the style function and add it to the map
#     geojson_layer = folium.GeoJson(
#         gdf[['zip_code','geometry', column_name]].dropna().to_json(),
#         name=column_name,
#         style_function=style_function,
#         tooltip=folium.GeoJsonTooltip(fields=['zip_code', column_name], aliases=['ZIP Code:', column_name])
#     )
    
#     # Add the GeoJson layer to a FeatureGroup
#     fg = folium.FeatureGroup(name=column_name)
#     geojson_layer.add_to(fg)
    
#     # Add the FeatureGroup to the map object
#     fg.add_to(map_object)
    
#     # Add the colormap to the map
#     colormap.add_to(map_object)

# # Loop over the columns you want to visualize as separate layers
# for column in ['5 Year HPI Change', '1 year forecast (23-24)',
#        '2 year forecast (23-25)', '3 year forecast (23-26)',
#        '4 year forecast (23-27)', '5 year forecast (23-28)']:  # Replace with your actual column names
#     add_layer(gdf, column, m)

# # Add layer control to toggle layers
# folium.LayerControl().add_to(m)

# # Display the map
# m

In [69]:
gdf.columns

Index(['County', '1 year forecast', '2 year forecast', '3 year forecast',
       '4 year forecast', '5 year forecast', 'OBJECTID', 'NAME', 'STATE_NAME',
       'STATE_FIPS', 'FIPS', 'SQMI', 'POPULATION', 'POP_SQMI', 'STATE_ABBR',
       'COUNTY_FIPS', 'geometry'],
      dtype='object')

In [71]:
gdf['5 year forecast'].min()

'15.80%'

In [116]:
gdf['5 year forecast'].max()

'29.20%'

## Map URL Snagger

In [75]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/FloridaCountyHPIforecast
